# Notebook 6 : MongoDB

In [ ]:
# Décommenter la ligne suivante pour installer pymongo
# %pip install pymongo

In [ ]:
import json

import pandas as pd
import pymongo

client = pymongo.MongoClient(
    # Coller ici la configuration donnée dans Onyxia
)

db = client.defaultdb

## Planètes de Star Wars

Nous considérons ici les données des planètes de *Star Wars* exportées à la fin du *Notebook 4*. Le fichier `planets.json` est également disponible dans le dossier des jeux de données.

1. Accéder à une collection `planets` et s'assurer qu'elle est vide grâce à la méthode `count_documents`.

2. Importer les données des planètes dans la collection `planets`.

3. Exporter l'ensemble des planètes sans l'identifiant `_id` dans un dataframe à l'aide du résultat de la méthode `find`.

4. Rechercher les planètes dont la période de rotation est égale à 25. Quel est le problème ? Combien y en a-t-il ?

5. Même question mais en limitant la réponse aux clés `name`, `rotation_period`, `orbital_period` et `diameter`.

6. Trier les planètes du résultat précédent par diamètre décroissant. Quel est le problème ?

7. Vider la collection et importer à nouveau les données mais en faisant les corrections suivantes au préalable (un dataframe intermédiaire pourra être utilisé pour manipuler les données avant leur insertion) :
- convertir les valeurs numériques (gérer les cas `unknown`),
- supprimer les variables `created`, `edited`, `films`, `gravity`, `residents` et `url`.
- transformer les variables `climate` et `terrain` en listes de chaînes de caractères plutôt qu'une longue chaîne séparée par des virgules.

8. Reprendre la question 6 et vérifier que le résultat est maintenant correct.

9. Extraire les planètes dont le nom commence par `T`.

10. Extraire les planètes dont le diamètre est strictement supérieur à 10000 et où se trouvent des montagnes.

11. Rechercher puis supprimer la planète dont le nom est `unknown`.

12. Mettre en œuvre un pipeline d'agrégation qui calcule le nombre de planètes dans la collection. Verifier le résultat avec la méthode `count_documents`.

13. Mettre en œuvre un pipeline d'agrégation pour calculer le diamètre moyen et la somme des populations des planètes contenant des glaciers.